### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()
nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
type(nyt_api_key)
type(tmdb_api_key)



str

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
req_url = (f"{url}api-key={nyt_api_key}"
             f"&fq={filter_query}"
             f"&sort={sort}"
             f"&fl={field_list}"
             f"&begin_date={begin_date}"
             f"&end_date={end_date}")


In [4]:
# Create an empty list to store the reviews
data_review_NYT = []

# loop through pages 0-19
for page in range(20):
    # Make a "GET" request and retrieve the JSON
    # create query with a page number
    # API results show 10 articles at a time
    reqst_url = (f"{url}api-key={nyt_api_key}"
                 f"&fq={filter_query}"
                 f"&sort={sort}"
                 f"&fl={field_list}"
                 f"&begin_date={begin_date}"
                 f"&end_date={end_date}"
                 f"&page={page}")
    response = requests.get(reqst_url)
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] 
        # and append each review to the list
        data = response.json()
        docs = data["response"]["docs"]
        # Print the page number that had no results then break from the loop
        if not docs:
            print(f"Page {page} had no results")
            break
        for doc in docs:
            data_review_NYT.append(doc)
        # Print the page that was just retrieved 
        print(f"Checked page {page}")
    #basic error handling    
    except KeyError:
        print(f"Error receiving data for page {page}")
# validation purpose
print(len(data_review_NYT))

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19
200


In [5]:
# Preview the first 5 results in JSON format
if page:
# Use json.dumps with argument indent=4 to format data
    print(json.dumps(data_review_NYT[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
ny_df = json_normalize(data_review_NYT)
ny_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
def extract_title(headline):
    # Title is between unicode characters \u2018 and \u2019. 
    start = headline.find('\u2018') + 1
    end = headline.find('\u2019', start)
    return headline[start:end] if start != -1 and end != -1 else headline
# save it to a new column "title"
# End string should include " Review" to avoid cutting title early
ny_df['title'] = ny_df['headline.main'].apply(extract_title)
ny_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    #to convert cells from a list to a string
    #we check (condition) if parameter of the function is a [list]
    #then we convt to a str if parameter is a [list]
    if isinstance(keyword_list, str):
        return keyword_list
    elif isinstance(keyword_list, list):
        extracted_keywords = ""
        for item in keyword_list:
        # Extract 'name' and 'value'
            keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
            extracted_keywords += keyword
        return extracted_keywords
    else:
        return""

# Fix the "keywords" column by converting cells from a list to a string
ny_df['keywords'] = ny_df['keywords'].apply(extract_keywords)
ny_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [11]:
# Create a list from the "title" column using to_list()
title_db = ny_df['title'].to_list()
# These titles will be used in the query for The Movie Database
title_db



['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love 

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [13]:
# Create an empty list to store the results
movie_lst = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
# Loop through the titles
# Check if we need to sleep before making a request
    # Add 1 to the request counter
try:
    #Enumerate method utilzed for request counter
    for idx, title in enumerate(title_db):
        if idx % 50 == 0 and idx != 0:
            time.sleep(50)
    # Perform a "GET" request for The Movie Database
    # Include a try clause to search for the full movie details.
        search_url = f"{url}{title}{tmdb_key_string}"
    # Use the except clause to print out a statement if a movie
        try:
            response = requests.get(search_url)
            response.raise_for_status()
            movie_search = response.json()
            # is not found.
            if not movie_search['results']:
                print(f"No movie found for title: {title}")
                continue
            # Get movie id
            movie_id = movie_search['results'][0]['id']


            # Make a request for a the full movie details
            movie_detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"


            # Execute "GET" request with url
            detail_response = requests.get(movie_detail_url)
            detail_response.raise_for_status()
            movie_details = detail_response.json()

        
            # Extract the genre names into a list
            genres = [genre['name'] for genre in movie_details['genres']]


            # Extract the spoken_languages' English name into a list
            languages = [lang['english_name'] for lang in movie_details['spoken_languages']]


            # Extract the production_countries' name into a list
            countries = [country['name'] for country in movie_details['production_countries']]
            # Add the relevant data to a dictionary and
                # append it to the tmdb_movies_list list
            movie_lst.append({
                'title': title,
                'genres': genres,
                'languages': languages,
                'countries': countries,
                'release_date': movie_details.get('release_date', ''),
                'runtime': movie_details.get('runtime', 0),
                'vote_average': movie_details.get('vote_average', 0),
                'vote_count': movie_details.get('vote_count', 0)
            })
            # Print out the title that was found
            print(f"Found movie {title}")
            #JSON Decode Error 
            # utilized to re-check the validity of the JSON data that was parsed
        except (json.JSONDecodeError, KeyError):
            print(f"Movie not found")
            #error handling for the status of the HTTP req
        except requests.RequestException as e:
            print(f"Request failed to retrieve title")
            #keyboard interrupt when running away from desk
except KeyboardInterrupt:
    print("Process terminated to prevent Ddos")




Found movie The Attachment Diaries
Found movie What
Found movie You Can Live Forever
Found movie A Tourist
Found movie Other People
Found movie One True Loves
Found movie The Lost Weekend: A Love Story
Found movie A Thousand and One
Found movie Your Place or Mine
Found movie Love in the Time of Fentanyl
Found movie Pamela, a Love Story
Found movie In From the Side
Found movie After Love
Found movie Alcarràs
Found movie Nelly & Nadine
Found movie Lady Chatterley
Found movie The Sound of Christmas
Found movie The Inspection
Found movie Bones and All
Found movie My Policeman
Found movie About Fate
Found movie Waiting for Bojangles
Found movie I Love My Dad
Found movie A Love Song
Found movie Alone Together
Found movie Art of Love
Found movie The Wheel
Found movie Thor: Love and Thunder
Found movie Both Sides of the Blade
Found movie Fire of Love
Found movie Love & Gelato
Found movie Stay Prayed Up
Found movie Benediction
Found movie Dinner in America
Found movie In a New York Minute
Found

In [15]:
# Preview the first 5 results in JSON format
if title_db:
# Use json.dumps with argument indent=4 to format data
    print(json.dumps(movie_lst[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "languages": [
            "Spanish"
        ],
        "countries": [
            "Argentina"
        ],
        "release_date": "2021-10-07",
        "runtime": 102,
        "vote_average": 3.0,
        "vote_count": 4
    },
    {
        "title": "What",
        "genres": [],
        "languages": [],
        "countries": [],
        "release_date": "",
        "runtime": 0,
        "vote_average": 0.0,
        "vote_count": 0
    },
    {
        "title": "You Can Live Forever",
        "genres": [
            "Drama",
            "Romance"
        ],
        "languages": [
            "English",
            "French"
        ],
        "countries": [
            "Canada",
            "United States of America"
        ],
        "release_date": "2023-03-24",
        "runtime": 96,
        "vote_average": 6

In [16]:
# Convert the results to a DataFrame 
tmb_df = pd.DataFrame(movie_lst)
tmb_df


,title,genres,languages,countries,release_date,runtime,vote_average,vote_count
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,3.000,4
1,What,[],[],[],,0,0.000,0
2,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,6.600,37
3,A Tourist,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,6.287,162
4,Other People,"[Comedy, Drama]",[English],[United States of America],2016-09-09,97,6.300,193
...,...,...,...,...,...,...,...,...
193,The Other Half,"[Drama, Romance]",[English],[Canada],2016-12-02,103,6.300,24
194,The Ottoman Lieutenant,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]",2017-03-10,111,6.164,247
195,Love & Taxes,"[Romance, Action, Drama]",[Turkish],[Turkey],2024-03-13,99,6.527,112
196,"Everybody Loves Somebody,","[Romance, Comedy]","[English, Spanish]",[Mexico],2017-02-10,102,6.538,159


### Merge and Clean the Data for Export

In [17]:
# Merge the New York Times reviews and TMDB DataFrames on title 
merge_df = pd.merge(ny_df, tmb_df, on='title', how='inner')
merge_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.person,byline.organization,title,genres,languages,countries,release_date,runtime,vote_average,vote_count
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,3.000,4
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,"[{'firstname': 'Jeannette', 'middlename': None...",None,What,[],[],[],,0,0.000,0
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,6.600,37
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,6.287,162
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People,"[Comedy, Drama]",[English],[United States of America],2016-09-09,97,6.300,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,...,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half,"[Drama, Romance]",[English],[Canada],2016-12-02,103,6.300,24
198,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",...,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]",2017-03-10,111,6.164,247
199,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",...,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes,"[Romance, Action, Drama]",[Turkish],[Turkey],2024-03-13,99,6.527,112
200,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up her

In [18]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
col_to_fix = ['headline.main', 'byline.person']

# Create a list of characters to remove
chrt_rm = ["[", "]", "'", "\""]
# Loop through the list of columns to fix
for col in col_to_fix:
    # Convert the column to type 'str'
    merge_df[col] = merge_df[col].astype(str)


    # Loop through characters to remove
    for char in chrt_rm:
        merge_df[col] = merge_df[col].str.replace(char, "", regex=False)


# Display the fixed DataFrame
merge_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.person,byline.organization,title,genres,languages,countries,release_date,runtime,vote_average,vote_count
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"{firstname: Jeannette, middlename: None, lastn...",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,3.000,4
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,"{firstname: Jeannette, middlename: None, lastn...",None,What,[],[],[],,0,0.000,0
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"{firstname: Elisabeth, middlename: None, lastn...",None,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,6.600,37
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"{firstname: Elisabeth, middlename: None, lastn...",None,A Tourist,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,6.287,162
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"{firstname: Manohla, middlename: None, lastnam...",None,Other People,"[Comedy, Drama]",[English],[United States of America],2016-09-09,97,6.300,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,...,"{firstname: Andy, middlename: None, lastname: ...",None,The Other Half,"[Drama, Romance]",[English],[Canada],2016-12-02,103,6.300,24
198,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",...,"{firstname: Neil, middlename: None, lastname: ...",None,The Ottoman Lieutenant,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]",2017-03-10,111,6.164,247
199,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",...,"{firstname: Ken, middlename: None, lastname: J...",None,Love & Taxes,"[Romance, Action, Drama]",[Turkish],[Turkey],2024-03-13,99,6.527,112
200,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up her

In [19]:
# Drop "byline.person" column
merge_df.drop(columns=["byline.person"], inplace=True)


In [21]:
# Delete duplicate rows and reset index
merge_df = merge_df.applymap(lambda x: str(x) if isinstance(x, list) else x)
merge_df = merge_df.drop_duplicates().reset_index(drop=True)
tmb_df


C:\Users\Richard Mahmud Okhai\AppData\Local\Temp\ipykernel_10272\4120142223.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merge_df = merge_df.applymap(lambda x: str(x) if isinstance(x, list) else x)


,title,genres,languages,countries,release_date,runtime,vote_average,vote_count
0,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,3.000,4
1,What,[],[],[],,0,0.000,0
2,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,6.600,37
3,A Tourist,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,6.287,162
4,Other People,"[Comedy, Drama]",[English],[United States of America],2016-09-09,97,6.300,193
...,...,...,...,...,...,...,...,...
193,The Other Half,"[Drama, Romance]",[English],[Canada],2016-12-02,103,6.300,24
194,The Ottoman Lieutenant,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]",2017-03-10,111,6.164,247
195,Love & Taxes,"[Romance, Action, Drama]",[Turkish],[Turkey],2024-03-13,99,6.527,112
196,"Everybody Loves Somebody,","[Romance, Comedy]","[English, Spanish]",[Mexico],2017-02-10,102,6.538,159


In [43]:
# Export data to CSV without the index
tmb_df.to_csv("movie_data.csv", encoding="utf-8", index=False)